# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [2]:
from azureml.core import Workspace, Experiment, Dataset, Datastore
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

from azureml.core import Experiment, Workspace, Environment, ScriptRunConfig
from azureml.core.conda_dependencies import CondaDependencies
from azureml.train.hyperdrive import HyperDriveConfig, PrimaryMetricGoal
from azureml.train.hyperdrive import BanditPolicy, BayesianParameterSampling, RandomParameterSampling, choice, quniform
from azureml.train.hyperdrive import HyperDriveRunConfig
from azureml.widgets import RunDetails

## Dataset

The same dataset using the automl experiment is going to be used for the Hyperdrive experiment.
The dataset is loaded in the train.py script and processed so it can be used as an Input to the model

In [4]:
ws = Workspace.from_config()
experiment_name = 'capstone-project'

experiment=Experiment(ws, experiment_name)

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [4]:
list(range(0.01,11))

[3, 4, 5, 6, 7, 8, 9, 10]

### setting up compute cluster

In [7]:
# Choose a name for your CPU cluster
cpu_cluster_name = "capstone-cluster2"

# Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    # To use a different region for the compute, add a location='<region>' parameter
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                            min_nodes=1,
                                                           max_nodes=6)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [8]:
# TODO: Create an early termination policy. This is not required if you are using Bayesian sampling.
early_termination_policy = BanditPolicy(evaluation_interval=1, slack_factor=0.1)

# Create the different params that you will be using during training
param_sampling = param_sampling = RandomParameterSampling({
    '--max_depth': choice([3, 4, 5, 6, 7, 8, 9, 10]),
    '--n_estimators': choice([100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]),
    '--_lambda': choice([0.001, 0.01, 0.1, 1, 10, 100]),
    '--subsample': quniform(min_value=0.5, max_value=1, q=10),
    '--colsample_bytree': quniform(min_value=0.6, max_value=1, q=10)
})

# setting up environment for the train.py script
env = Environment(name="capstone-hyperdrive-env")

conda_dep = CondaDependencies()
conda_dep.add_conda_package("scikit-learn")
conda_dep.add_conda_package("numpy")
conda_dep.add_pip_package("pandas")
conda_dep.add_conda_package("xgboost")
conda_dep.add_pip_package("azureml-sdk")
conda_dep.add_pip_package("joblib")
env.python.conda_dependencies = conda_dep

# Creating a ScriptRunConfig instead of and Estimator, because these have been deprecated
src = script_run_config = ScriptRunConfig(
    source_directory='.',
    script='train.py',
    compute_target=cpu_cluster,
    environment=env
)

# Creating HyperdriveConfig
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=param_sampling,
                                     max_total_runs=1000,
                                     policy=early_termination_policy,
                                     primary_metric_name='NRMSE',
                                     primary_metric_goal=PrimaryMetricGoal.MINIMIZE,
                                     max_duration_minutes=30,
                                     max_concurrent_runs=6)

In [9]:
# Submit your experiment
hyperdrive_run = experiment.submit(hyperdrive_config)

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [10]:
RunDetails(hyperdrive_run).show()

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [ ]:
#TODO: Save the best model

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

